In [ ]:
!pip install pingouin

# Normalization 

In [ ]:
import pandas as pd
feature_matrix_pt = pd.read_csv("/kaggle/input/raw-data/output_with_params_pt.csv")
feature_matrix_pr = pd.read_csv("/kaggle/input/raw-data/output_with_params_pr.csv")

In [ ]:
feature_matrix_pr

In [ ]:
# 1. Создаем столбец с длиной текста в словах
def get_text_length(text):
    if isinstance(text, str):
        return len(text.split())
    else:
        return 0  # Обработка NaN и нестроковых значений

# Применяем функцию к столбцу с текстом
feature_matrix_pt['text_length'] = feature_matrix_pt['Текст открытки'].apply(get_text_length)
feature_matrix_pr['text_length'] = feature_matrix_pr['text'].apply(get_text_length)

In [ ]:
feature_matrix_pt.drop(
    feature_matrix_pt[feature_matrix_pt['text_length'] <= 2].index,
    inplace=True
)
feature_matrix_pt.reset_index(drop=True, inplace=True)


feature_matrix_pr.drop(
    feature_matrix_pr[feature_matrix_pr['text_length'] <= 2].index,
    inplace=True
)
feature_matrix_pr.reset_index(drop=True, inplace=True)

In [ ]:
# 2. Определим, какие колонки являются признаками 
meta_columns = ['text', 'year', 'decade', 'text_length']
feature_columns = [col for col in feature_matrix_pr.columns if col not in meta_columns]

In [ ]:
import numpy as np

# Признаки, которые НЕ нужно нормализовать (уже индексные, средние и т.п.)
potentially_problematic_columns = [
    'noun_abstr_index_abs',
    'adj_abstr_index_abs',
    'flesch_kincaid_index_abs',
    'avg_vp_length_abs',
    'avg_np_length_abs',
    'max_tree_depth_abs',
    'word length_abs',
    'type-token ratio_abs',
    'mean_sentence_length_abs'
]

# Копируем матрицу признаков
normalized_feature_matrix_pt = feature_matrix_pt.copy()

# Удаляем строки с подозрительно высокими частотами (только для частотных признаков)
rows_to_keep = []
for index, row in feature_matrix_pt.iterrows():
    keep_row = True
    for feature in feature_columns:
        if feature not in potentially_problematic_columns:
            try:
                dif = row[feature] / (row['text_length'] + 1)
                if dif > 1:
                    keep_row = False
                    break
            except ZeroDivisionError:
                keep_row = False
                break
    if keep_row:
        rows_to_keep.append(index)

# Оставляем только нормальные строки
normalized_feature_matrix_pt = normalized_feature_matrix_pt.iloc[rows_to_keep]

# Применяем нормализацию по Байберу ТОЛЬКО к частотным признакам
for feature in feature_columns:
    if feature not in potentially_problematic_columns:
        normalized_feature_matrix_pt[feature] = (
            normalized_feature_matrix_pt[feature] / (normalized_feature_matrix_pt['text_length'] + 1) * 100
        ).replace([np.inf, -np.inf], np.nan).round(3)
    else:
        # Оставляем как есть
        normalized_feature_matrix_pt[feature] = normalized_feature_matrix_pt[feature]

# Сброс индекса
normalized_feature_matrix_pt = normalized_feature_matrix_pt.reset_index(drop=True)

# Готово
normalized_feature_matrix_pt


In [ ]:
import numpy as np

# Признаки, которые НЕ нужно нормализовать (уже индексные, средние и т.п.)
potentially_problematic_columns = [
    'noun_abstr_index_abs',
    'adj_abstr_index_abs',
    'flesch_kincaid_index_abs',
    'avg_vp_length_abs',
    'avg_np_length_abs',
    'max_tree_depth_abs',
    'word length_abs',
    'type-token ratio_abs',
    'mean_sentence_length_abs'
]

# Копируем матрицу признаков
normalized_feature_matrix_pr = feature_matrix_pr.copy()

# Удаляем строки с подозрительно высокими частотами (только для частотных признаков)
rows_to_keep = []
for index, row in feature_matrix_pr.iterrows():
    keep_row = True
    for feature in feature_columns:
        if feature not in potentially_problematic_columns:
            try:
                dif = row[feature] / (row['text_length'] + 1)
                if dif > 1:
                    keep_row = False
                    break
            except ZeroDivisionError:
                keep_row = False
                break
    if keep_row:
        rows_to_keep.append(index)

# Оставляем только нормальные строки
normalized_feature_matrix_pr = normalized_feature_matrix_pr.iloc[rows_to_keep]

# Применяем нормализацию по Байберу ТОЛЬКО к частотным признакам
for feature in feature_columns:
    if feature not in potentially_problematic_columns:
        normalized_feature_matrix_pr[feature] = (
            normalized_feature_matrix_pr[feature] / (normalized_feature_matrix_pr['text_length'] + 1) * 100
        ).replace([np.inf, -np.inf], np.nan).round(3)
    else:
        # Оставляем как есть
        normalized_feature_matrix_pr[feature] = normalized_feature_matrix_pr[feature]

# Сброс индекса
normalized_feature_matrix_pr = normalized_feature_matrix_pr.reset_index(drop=True)

# Готово
normalized_feature_matrix_pr


In [ ]:
normalized_feature_matrix_pr.to_csv('normalized_feature_matrix_pt.csv')
normalized_feature_matrix_pr.to_csv('normalized_feature_matrix_pr.csv')

In [ ]:
feature_columns = [col for col in normalized_feature_matrix_pr.columns if col not in ['Текст открытки', 'decade', 'год', 'text_length', 'Unnamed: 0', 'year', 'text', 'other_coordination_abs', 'type']]
len(feature_columns)

In [ ]:
for col in feature_columns:
        normalized_feature_matrix_pr[col] = normalized_feature_matrix_pr[col].fillna(0).astype('float64')

nan_cols = normalized_feature_matrix_pr.columns[normalized_feature_matrix_pr.isnull().any()]
print("Столбцы, содержащие NaN:", nan_cols)

for col in feature_columns:
        normalized_feature_matrix_pr[col] = normalized_feature_matrix_pr[col].fillna(0).astype('float64')

nan_cols = normalized_feature_matrix_pr.columns[normalized_feature_matrix_pr.isnull().any()]
print("Столбцы, содержащие NaN:", nan_cols)

In [ ]:
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests
import pingouin as pg

def analyze_large_corpora(corpus1_df, corpus2_df, features, chunk_size=10000):
    """
    Анализ предзагруженных корпусов
    
    Параметры:
    corpus1_df, corpus2_df - предзагруженные DataFrame
    features - список признаков для анализа
    chunk_size - размер чанка для обработки (по умолчанию 10,000)
    """
    
    # Инициализация результатов
    results = {feature: {'mean1': 0.0, 'mean2': 0.0, 'd': 0.0, 'p': 1.0} 
               for feature in features}
    
    # Обработка корпуса 1
    total1 = 0
    for i in range(0, len(corpus1_df), chunk_size):
        chunk = corpus1_df.iloc[i:i+chunk_size]
        total1 += len(chunk)
        for feature in features:
            results[feature]['mean1'] += chunk[feature].sum()

    # Обработка корпуса 2
    total2 = 0
    for i in range(0, len(corpus2_df), chunk_size):
        chunk = corpus2_df.iloc[i:i+chunk_size]
        total2 += len(chunk)
        for feature in features:
            results[feature]['mean2'] += chunk[feature].sum()

    # Расчет финальных средних
    for feature in features:
        results[feature]['mean1'] /= total1 if total1 > 0 else 1
        results[feature]['mean2'] /= total2 if total2 > 0 else 1

    # Расчет статистик
    p_values = []
    cohens_d_values = []
    for feature in features:
        # Данные для признака
        data1 = corpus1_df[feature].values
        data2 = corpus2_df[feature].values
        
        # t-тест Уэлча
        try:
            _, p = ttest_ind(data1, data2, equal_var=False, nan_policy='omit')
        except:
            p = 1.0
        
        # Cohen's d
        try:
            # n1, n2 = len(data1), len(data2)
            # var1 = np.nanvar(data1, ddof=1)
            # var2 = np.nanvar(data2, ddof=1)
            # pooled_var = ((n1-1)*var1 + (n2-1)*var2) / (n1 + n2 - 2)
            # pooled_std = np.sqrt(pooled_var) if pooled_var > 0 else 0
            # d = (np.nanmean(data1) - np.nanmean(data2)) / pooled_std if pooled_std != 0 else 0
            d = pg.compute_effsize(data1, data2, eftype='cohen')
        except:
            d = 0
        
        p_values.append(p)
        cohens_d_values.append(d)

    # Поправка FDR
    p_values = [1.0 if np.isnan(p) else p for p in p_values]
    _, p_corrected, _, _ = multipletests(p_values, alpha=0.05, method='fdr_bh')

    # Сбор результатов
    return pd.DataFrame({
        'feature': features,
        'mean_corpus1': [results[f]['mean1'] for f in features],
        'mean_corpus2': [results[f]['mean2'] for f in features],
        'p_value': p_values,
        'p_corrected': p_corrected,
        'cohens_d': cohens_d_values
    })

In [ ]:
# Анализ
results = analyze_large_corpora(normalized_feature_matrix_pt, normalized_feature_matrix_pr, feature_columns)
results

In [ ]:
# Фильтрация значимых результатов
significant = results[(results['p_corrected'] < 0.05) & (abs(results['cohens_d']) >= 0.5)]
significant

In [ ]:
normalized_feature_matrix_pr['type'] = 'diary'
normalized_feature_matrix_pt['type'] = 'postcard'

normalized_feature_matrix_pt.rename(columns={"Текст открытки": "text", "Год открытки": "year"}, inplace=True)
cols = normalized_feature_matrix_pr.columns.tolist()
cols[0], cols[1] = cols[1], cols[0]
normalized_feature_matrix_pr = normalized_feature_matrix_pr[cols]

In [ ]:
normalized_feature_matrix_pt.columns == normalized_feature_matrix_pr.columns

In [ ]:
def concatenate_dataframes(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
    
    if df1.columns.tolist() != df2.columns.tolist():
        print("Ошибка: DataFrame имеют разные колонки.")
        return None
    
    concatenated_df = pd.concat([df1, df2], ignore_index=True)
    return concatenated_df

In [ ]:
result_df = concatenate_dataframes(normalized_feature_matrix_pt, normalized_feature_matrix_pr)

In [ ]:
result_df

# United factor analysis 

In [ ]:
counts = result_df.groupby(['decade', 'type']).size().unstack(fill_value=0)
print("Исходное распределение:\n", counts)

In [ ]:
min_count = (
    result_df.groupby(['decade', 'type']).size().min()
)

# Найдём только те декады, где есть достаточно и того, и другого
valid_decades = result_df.groupby(['decade', 'type']).size().unstack().dropna()
valid_decades = valid_decades[(valid_decades['diary'] >= min_count) & (valid_decades['postcard'] >= min_count)].index

# Теперь выборка
balanced_samples = []

for decade in valid_decades:
    for text_type in ['postcard', 'diary']:
        subset = result_df[(result_df['decade'] == decade) & (result_df['type'] == text_type)]
        sampled = subset.sample(n=min_count, random_state=42)
        balanced_samples.append(sampled)

df_balanced = pd.concat(balanced_samples, ignore_index=True)

print("\nРаспределение после выравнивания:\n", 
      df_balanced.groupby(['decade', 'type']).size().unstack())


In [ ]:
df_balanced

In [ ]:
pip install --upgrade scipy


In [ ]:
import pingouin as pg

result_num = result_df.select_dtypes(include='number')
balanced_num = df_balanced.select_dtypes(include='number')


# multivariate t-test для двух наборов данных
test = pg.multivariate_ttest(result_num, balanced_num)

print(test)


In [ ]:
# import pandas as pd
# import numpy as np
# from scipy.stats import ttest_ind, levene, ks_2samp
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Убедимся, что только числовые признаки
# numeric_cols = result_df.select_dtypes(include=[np.number]).columns.intersection(
#     df_balanced.select_dtypes(include=[np.number]).columns
# )

# # Для хранения результатов
# results = []

# for col in numeric_cols:
#     # t-test
#     t_stat, t_p = ttest_ind(result_df[col], df_balanced[col], equal_var=False, nan_policy='omit')
    
#     # Levene's test (на гомогенность дисперсий)
#     lev_stat, lev_p = levene(result_df[col], df_balanced[col], center='median')
    
#     # K-S test (на одинаковость распределений)
#     ks_stat, ks_p = ks_2samp(result_df[col].dropna(), df_balanced[col].dropna())
    
#     results.append({
#         'feature': col,
#         't_pvalue': t_p,
#         'levene_pvalue': lev_p,
#         'ks_pvalue': ks_p
#     })

# # В DataFrame
# results_df = pd.DataFrame(results)

# # Сколько признаков НЕ имеют значимых различий (p > 0.05)
# no_diff_t = (results_df['t_pvalue'] > 0.05).sum()
# no_diff_ks = (results_df['ks_pvalue'] > 0.05).sum()

# print(f"Без значимых различий по t-тесту: {no_diff_t}/{len(numeric_cols)} признаков")
# print(f"Без значимых различий по K-S тесту: {no_diff_ks}/{len(numeric_cols)} признаков")

# # Показать признаки с наиболее сильными различиями
# print(results_df.nsmallest(10, 'ks_pvalue'))


In [ ]:
# # Можно выбрать случайные 5 признаков для графиков

# sample_cols = np.random.choice(numeric_cols, size=5, replace=False)

# for col in sample_cols:
#     plt.figure(figsize=(8, 4))
#     sns.kdeplot(result_df[col], label='Original', fill=True, alpha=0.5)
#     sns.kdeplot(df_balanced[col], label='Balanced', fill=True, alpha=0.5)
#     plt.title(f"Распределение признака: {col}")
#     plt.legend()
#     plt.show()


In [ ]:
descriptive_stats = df_balanced.describe().T[['mean', 'min', 'max', 'std']]
descriptive_stats['range'] = descriptive_stats['max'] - descriptive_stats['min']

# Сохраняем в таблицу
descriptive_stats.to_csv('descriptive_stats.csv')
print(descriptive_stats)

In [ ]:
# 1. Сначала создаем копию normalized_feature_matrix:
fa_matrix = df_balanced.copy(deep=True)

fa_matrix = fa_matrix.drop(['year', 'text_length'], axis=1)

fa_matrix.head()


full_fa_matrix = result_df.copy(deep=True)
full_fa_matrix = full_fa_matrix.drop(['year', 'text_length'], axis=1)

fa_matrix.head()


In [ ]:
# Выбираем только числовые признаки (исключаем метаданные)
feature_columns = [col for col in fa_matrix.columns if col not in ['text', 'decade', 'type']]

# Создаем DataFrame только с признаками для факторного анализа
analysis_df = fa_matrix[feature_columns].copy(deep=True)
analysis_df_full = full_fa_matrix[feature_columns].copy(deep=True)

# Убедимся, что все данные числовые
print(analysis_df.dtypes)

In [ ]:
!pip install factor_analyzer

In [ ]:
nan_cols = analysis_df.columns[analysis_df.isnull().any()]
print("Столбцы, содержащие NaN:", nan_cols)

df_with_nan = analysis_df[nan_cols]

# 3. Создаем булеву маску, где True соответствует строкам, содержащим хотя бы один NaN
rows_with_nan = df_with_nan.isnull().any(axis=1)

# 4. Считаем количество строк, содержащих хотя бы один NaN
print(rows_with_nan.sum())

for col in nan_cols:
    analysis_df[col] = analysis_df[col].fillna(0).astype('float64')


print(analysis_df.isnull().sum())


In [ ]:
nan_cols = analysis_df_full.columns[analysis_df_full.isnull().any()]
print("Столбцы, содержащие NaN:", nan_cols)

df_with_nan = analysis_df_full[nan_cols]

# 3. Создаем булеву маску, где True соответствует строкам, содержащим хотя бы один NaN
rows_with_nan = df_with_nan.isnull().any(axis=1)

# 4. Считаем количество строк, содержащих хотя бы один NaN
print(rows_with_nan.sum())

for col in nan_cols:
    analysis_df_full[col] = analysis_df_full[col].fillna(0).astype('float64')


print(analysis_df_full.isnull().sum())


In [ ]:
stddevs = analysis_df.std()
print(stddevs[stddevs == 0])

In [ ]:
analysis_df = analysis_df.drop('other_coordination_abs', axis=1)
stddevs = analysis_df.std()
print(stddevs[stddevs == 0])

In [ ]:
stddevs = analysis_df_full.std()
print(stddevs[stddevs == 0])

In [ ]:
analysis_df_full = analysis_df_full.drop('other_coordination_abs', axis=1)
stddevs = analysis_df_full.std()
print(stddevs[stddevs == 0])

In [ ]:
feature_columns_new = [col for col in analysis_df.columns]
len(feature_columns_new)

In [ ]:
from factor_analyzer import FactorAnalyzer

# Инициализация PFA без вращения
fa = FactorAnalyzer(n_factors=len(feature_columns), rotation=None, method='principal')

# Подгонка модели
fa.fit(analysis_df[feature_columns_new])

# Собственные значения
ev, _ = fa.get_eigenvalues()

loadings_first = fa.loadings_

In [ ]:
from factor_analyzer import FactorAnalyzer

# Инициализация и обучение модели на всех данных
fa_full = FactorAnalyzer(
    n_factors=len(feature_columns),
    rotation=None,
    method='principal'
)

fa_full.fit(analysis_df_full[feature_columns_new])  # analysis_df — весь ваш DataFrame с данными

# Получение нагрузок
loadings_full = fa_full.loadings_  # размерность: (признаки, факторы)


In [ ]:
import numpy as np
import pandas as pd
from factor_analyzer import FactorAnalyzer
from tqdm import tqdm

# Исходные данные: df — DataFrame с числовыми переменными для факторного анализа

n_iterations = 10
sample_fraction = 0.07
n_factors = len(feature_columns)  # например, нужно 3 фактора

# Хранить факторные нагрузки для каждой переменной и фактора
all_loadings = []

def has_zero_variance(df):
    return any(df.std() == 0)

for i in tqdm(range(n_iterations)):
    # Бутстрэп-сэмпл
    sample = analysis_df.sample(frac=sample_fraction, replace=True, random_state=None)

    if sample.shape[0] <= n_factors or sample.shape[0] <= sample.shape[1]:
        # Пропустить итерацию, если строк мало
        continue

    if has_zero_variance(sample[feature_columns_new]):
        continue
    
    # Факторный анализ
    fa = FactorAnalyzer(n_factors=n_factors, rotation=None, method='principal')
    fa.fit(sample[feature_columns_new])
    
    # Получить факторные нагрузки
    loadings = fa.loadings_  # shape: (n_features, n_factors)
    all_loadings.append(loadings)

# Преобразуем в numpy массив: (n_iterations, n_features, n_factors)
all_loadings = np.array(all_loadings)

n_features = all_loadings.shape[1]

# Для каждой переменной и каждого фактора — доверительный интервал
for var_idx in range(n_features):
    for factor_idx in range(n_factors-1):
        # Получаем распределение нагрузок
        load_dist = all_loadings[:, var_idx, factor_idx]
        
        # Доверительный интервал 95%
        lower = np.percentile(load_dist, 2.5)
        upper = np.percentile(load_dist, 97.5)
        mean = np.mean(load_dist)
        
        print(f"Переменная {var_idx+1}, Фактор {factor_idx+1}: "
              f"Среднее = {mean:.3f}, 95% ДИ = [{lower:.3f}, {upper:.3f}]")


In [ ]:
import matplotlib.pyplot as plt
# График осыпи
plt.figure(figsize=(10, 6))
plt.scatter(range(1, len(ev)+1), ev)
plt.plot(range(1, len(ev)+1), ev)
plt.title('Scree Plot')
plt.xlabel('Номер фактора')
plt.ylabel('Собственное значение')
plt.axhline(y=1, color='r', linestyle='--')
plt.show()

# Таблица собственных значений и доли дисперсии
eigenvalues_table = pd.DataFrame({
    'Factor': range(1, len(ev)+1),
    'Eigenvalue': ev,
    '% of Variance': (ev / ev.sum()) * 100,
    'Cumulative %': (ev.cumsum() / ev.sum()) * 100
})

print(eigenvalues_table.head(11))  

In [ ]:
# Проверка наличия NaN и inf
print("Количество NaN в данных:", analysis_df.isna().sum().sum())
print("Количество inf в данных:", np.isinf(analysis_df.values).sum())

In [ ]:
# # Замена NaN и inf
# analysis_df_clean.fillna(0, inplace=True)
# analysis_df_clean.replace([np.inf, -np.inf], 0, inplace=True)

In [ ]:
import warnings

warnings.filterwarnings("ignore")


In [ ]:
# Инициализация модели с 9 факторами
fa_promax = FactorAnalyzer(n_factors=9, rotation='promax', method='principal')
fa_promax.fit(analysis_df[feature_columns_new])

# Факторные нагрузки
loadings = pd.DataFrame(
    fa_promax.loadings_,
    columns=[f'Factor {i+1}' for i in range(9)],
    index=feature_columns_new
)

# Сохраняем нагрузки в CSV
loadings.to_csv('factor_loadings.csv')

In [ ]:
# Выводим значимые нагрузки 
significant_loadings = loadings.applymap(lambda x: x if abs(x) > 0.3 else None)
# significant_loadings.dropna(how='all')

for factor in significant_loadings.columns:
    print(f'Нагрузки для {factor}:')

    # Берём нагрузки для текущего фактора и исключаем NaN
    for feature, loading in significant_loadings[factor].items():
        if not pd.isna(loading):
            print(f'  - {feature}: {loading}')
    print()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Получаем факторные корреляции
factor_corr = pd.DataFrame(
    fa_promax.phi_,
    columns=[f'Factor {i+1}' for i in range(9)],
    index=[f'Factor {i+1}' for i in range(9)]
)

# Визуализация
plt.figure(figsize=(10, 8))
sns.heatmap(factor_corr, annot=True, cmap='gray', vmin=-1, vmax=1)
plt.title('Межфакторные корреляции (Promax)')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Получаем факторные корреляции
factor_corr = pd.DataFrame(
    fa_promax.phi_,
    columns=[f'Factor {i+1}' for i in range(9)],
    index=[f'Factor {i+1}' for i in range(9)]
)

# Визуализация
plt.figure(figsize=(10, 8))
sns.heatmap(factor_corr, annot=True, cmap='gray', vmin=-1, vmax=1)
plt.title('Межфакторные корреляции (Promax)')
plt.show()

In [ ]:
loadings = pd.read_csv('/kaggle/input/united-loading-final/factor_loadings (3).csv', index_col='Unnamed: 0')

In [ ]:
# Порог значимости для включения признаков
threshold = 0.30

# Находим абсолютные нагрузки
loadings_abs = loadings.abs()

# Индексы факторов с максимальной нагрузкой для каждого признака
max_index = loadings_abs.idxmax(axis=1)

# Создаем словарь для хранения признаков по факторам
factor_dict = {}

# Заполняем словарь, выбирая только те нагрузки, которые выше порога
for feature in loadings.index:
    factor = max_index[feature]
    max_loading = loadings_abs.loc[feature, factor]

    if max_loading > threshold:
        if factor not in factor_dict:
            factor_dict[factor] = []
        factor_dict[factor].append([feature, float(loadings.loc[feature, factor])])

factor_dict


In [ ]:
# Порог значимости для включения признаков
threshold = 0.30

# Находим абсолютные нагрузки
loadings_abs = loadings.abs()

# Индексы факторов с максимальной нагрузкой для каждого признака
max_index = loadings_abs.idxmax(axis=1)

# Создаем словарь для хранения признаков по факторам
factor_dict = {}

# Заполняем словарь, выбирая только те нагрузки, которые выше порога
for feature in loadings.index:
    factor = max_index[feature]
    max_loading = loadings_abs.loc[feature, factor]

    if max_loading > threshold:
        if factor not in factor_dict:
            factor_dict[factor] = []
        factor_dict[factor].append([feature, float(loadings.loc[feature, factor])])

factor_dict


# Factor scores

In [ ]:
nan_cols = df_balanced.columns[df_balanced.isnull().any()]
print("Столбцы, содержащие NaN:", nan_cols)



In [ ]:
for col in nan_cols:
    df_balanced[col] = df_balanced[col].fillna(0).astype('float64')

nan_cols = df_balanced.columns[df_balanced.isnull().any()]
print("Столбцы, содержащие NaN:", nan_cols)

df_with_nan = df_balanced[nan_cols]

# Создаем булеву маску, где True соответствует строкам, содержащим хотя бы один NaN
rows_with_nan = df_with_nan.isnull().any(axis=1)

# Считаем количество строк, содержащих хотя бы один NaN
print(rows_with_nan.sum())


In [ ]:
standardized_feature_matrix  = df_balanced.copy()
standardized_feature_matrix

In [ ]:
standardized_feature_matrix  = standardized_feature_matrix.drop(['text', 'year', 'decade', 'type', 'text_length'], axis=1)
standardized_feature_matrix

In [ ]:
# descriptive_stats = pd.read_csv('/kaggle/input/pt-descrip-stats/descriptive_stats_pt.csv', index_col='Unnamed: 0')

In [ ]:
for column in standardized_feature_matrix.columns:
    mean = descriptive_stats.loc[column, 'mean']
    std = descriptive_stats.loc[column, 'std']

    standardized_feature_matrix[column] = (standardized_feature_matrix[column] - mean) / std
standardized_feature_matrix

In [ ]:
feature_columns_new = [col for col in standardized_feature_matrix.columns]
len(feature_columns_new)

In [ ]:
# Среднее после стандартизации должно быть ~0
print(standardized_feature_matrix[feature_columns_new].mean().round(2))  

# Стандартное отклонение должно быть ~1
print(standardized_feature_matrix[feature_columns_new].std().round(2))   

In [ ]:
# 1. Создаем DataFrame factor_scores с текстами и декадами
factor_scores = df_balanced[['text', 'decade', 'type']].copy()

# 2. Связываем standardized_feature_matrix с factor_scores по индексам
standardized_features = standardized_feature_matrix.copy()
standardized_features.index = factor_scores.index

In [ ]:
# 3. Функция для расчета факторной оценки
def calculate_factor_score(row, factor_features):
    score = 0
    for feature, loading in factor_features:
        # Получаем стандартизированное значение признака
        value = row[feature]
        # Учитываем знак нагрузки: + если loading > 0, - если loading < 0
        score += value * (1 if loading > 0 else -1)
    return score

In [ ]:
# 4. Для каждого фактора из factor_dict вычисляем оценку
for factor, features in factor_dict.items():
    # Признаки и их нагрузки для текущего фактора
    factor_features = [(feat[0], feat[1]) for feat in features]

    # Проверяем наличие признаков в standardized_features
    missing = [feat[0] for feat in features if feat[0] not in standardized_features.columns]
    if missing:
        print(f"Предупреждение: Признаки {missing} отсутствуют в standardized_feature_matrix.")
        continue

    # Рассчитываем факторную оценку
    factor_scores[factor] = standardized_features.apply(
        lambda row: calculate_factor_score(row, factor_features), axis=1
    )

factor_scores.head()

In [ ]:
factor_scores.to_csv('factor_scores.csv')

# Text by factors

In [ ]:
import pandas as pd
factor_scores = pd.read_csv('/kaggle/input/normal-data/factor_scores.csv', index_col = 'Unnamed: 0')

In [ ]:
factor_scores

In [ ]:
# Сортируем по убыванию Factor 1 и берем первые 10 строк
top_10 = factor_scores.sort_values(by='Factor 1', ascending=False)[:11]

# Выводим тексты и значения, включая тип текста
print("Тексты с наибольшими значениями Factor 1:")
for index, row in top_10.iterrows():
    print(f"Type: {row['type']}, Text: {row['text']}}}")

# diary_count = top_10[top_10['type'] == 'diary']['type'].count()
# postcard_count = top_10[top_10['type'] == 'postcard']['type'].count()

# print(f"Количество дневников в топ-100: {diary_count}")
# print(f"Количество открыток в топ-100: {postcard_count}")

In [ ]:
diary_count = top_10[top_10['type'] == 'diary']['type'].count()
postcard_count = top_10[top_10['type'] == 'postcard']['type'].count()

print(f"Количество дневников в топ-100: {diary_count}")
print(f"Количество открыток в топ-100: {postcard_count}")

In [ ]:
# Сортируем по возрастанию Factor 1 и берем первые 10 строк
bottom_10 = factor_scores.sort_values(by='Factor 1', ascending=True)[:20]

# Выводим тексты и значения
# Выводим тексты и значения, включая тип текста
print("Тексты с наименьшими значениями Factor 1:")
for index, row in bottom_10.iterrows():
    print(f"Type: {row['type']}, Text: {row['text']}}}")

# diary_count = bottom_10[bottom_10['type'] == 'diary']['type'].count()
# postcard_count = bottom_10[bottom_10['type'] == 'postcard']['type'].count()

# print(f"Количество дневников в топ-100: {diary_count}")
# print(f"Количество открыток в топ-100: {postcard_count}")

In [ ]:
diary_count = bottom_10[bottom_10['type'] == 'diary']['type'].count()
postcard_count = bottom_10[bottom_10['type'] == 'postcard']['type'].count()

print(f"Количество дневников в топ-100: {diary_count}")
print(f"Количество открыток в топ-100: {postcard_count}")

In [ ]:
# Сортируем по убыванию Factor 1 и берем первые 10 строк
top_10 = factor_scores.sort_values(by='Factor 2', ascending=False)[:10]

# Выводим тексты и значения, включая тип текста
print("Тексты с наибольшими значениями Factor 2:")
for index, row in top_10.iterrows():
    print(f"Type: {row['type']}, Text: {row['text']}}}")

# diary_count = top_10[top_10['type'] == 'diary']['type'].count()
# postcard_count = top_10[top_10['type'] == 'postcard']['type'].count()

# print(f"Количество дневников в топ-100: {diary_count}")
# print(f"Количество открыток в топ-100: {postcard_count}")

In [ ]:
diary_count = top_10[top_10['type'] == 'diary']['type'].count()
postcard_count = top_10[top_10['type'] == 'postcard']['type'].count()

print(f"Количество дневников в топ-100: {diary_count}")
print(f"Количество открыток в топ-100: {postcard_count}")

In [ ]:
# Сортируем по возрастанию Factor 1 и берем первые 10 строк
bottom_10 = factor_scores.sort_values(by='Factor 2', ascending=True)[:20]

# Выводим тексты и значения
# Выводим тексты и значения, включая тип текста
print("Тексты с наименьшими значениями Factor 2:")
for index, row in bottom_10.iterrows():
    print(f"Type: {row['type']}, Text: {row['text']}}}")

In [ ]:
# dif = feature_matrix_pr[feature_matrix_pr['text']==text]['avg_vp_length_abs'] / (feature_matrix_pr[feature_matrix_pr['text']==text]['text_length'] + 1)
# (dif) * 100

In [ ]:
# Сортируем по убыванию Factor 1 и берем первые 10 строк
top_10 = factor_scores.sort_values(by='Factor 3', ascending=False)[:100]

# Выводим тексты и значения, включая тип текста
print("Тексты с наибольшими значениями Factor 3:")
for index, row in top_10.iterrows():
    if row['type']=='postcard':
        print(f"Type: {row['type']}, Text: {row['text']}}}")



In [ ]:
diary_count = top_10[top_10['type'] == 'diary']['type'].count()
postcard_count = top_10[top_10['type'] == 'postcard']['type'].count()

print(f"Количество дневников в топ-100: {diary_count}")
print(f"Количество открыток в топ-100: {postcard_count}")

In [ ]:
# Сортируем по возрастанию Factor 1 и берем первые 10 строк
bottom_10 = factor_scores.sort_values(by='Factor 3', ascending=True)[:10]

# Выводим тексты и значения
# Выводим тексты и значения, включая тип текста
print("Тексты с наименьшими значениями Factor 3:")
for index, row in bottom_10.iterrows():
    print(f"Type: {row['type']}, Text: {row['text']}}}")

In [ ]:
diary_count = bottom_10[bottom_10['type'] == 'diary']['type'].count()
postcard_count = bottom_10[bottom_10['type'] == 'postcard']['type'].count()

print(f"Количество дневников в топ-100: {diary_count}")
print(f"Количество открыток в топ-100: {postcard_count}")

In [ ]:
# Сортируем по убыванию Factor 1 и берем первые 10 строк
top_10 = factor_scores.sort_values(by='Factor 4', ascending=False)[:100]

# Выводим тексты и значения, включая тип текста
print("Тексты с наибольшими значениями Factor 4:")
for index, row in top_10.iterrows():
    if row['type'] == 'postcard':
        print(f"Type: {row['type']}, Text: {row['text']}}}")



In [ ]:
# Сортируем по возрастанию Factor 1 и берем первые 10 строк
bottom_10 = factor_scores.sort_values(by='Factor 4', ascending=True)[:50]

# Выводим тексты и значения
# Выводим тексты и значения, включая тип текста
print("Тексты с наименьшими значениями Factor 4:")
for index, row in bottom_10.iterrows():
    print(f"Type: {row['type']}, Text: {row['text']}}}")

In [ ]:
# Сортируем по убыванию Factor 1 и берем первые 10 строк
top_10 = factor_scores.sort_values(by='Factor 5', ascending=False)[:500]

# Выводим тексты и значения, включая тип текста
print("Тексты с наибольшими значениями Factor 5:")
for index, row in top_10.iterrows():
    if row['type']=='diary':
        print(f"Type: {row['type']}, Text: {row['text']}}}")



In [ ]:
diary_count = top_10[top_10['type'] == 'diary']['type'].count()
postcard_count = top_10[top_10['type'] == 'postcard']['type'].count()

print(f"Количество дневников в топ-100: {diary_count}")
print(f"Количество открыток в топ-100: {postcard_count}")

In [ ]:
# Сортируем по возрастанию Factor 1 и берем первые 10 строк
bottom_10 = factor_scores.sort_values(by='Factor 5', ascending=True)[:200]

# Выводим тексты и значения
# Выводим тексты и значения, включая тип текста
print("Тексты с наименьшими значениями Factor 5:")
for index, row in bottom_10.iterrows():
    if row['type']=='postcard':
        print(f"Type: {row['type']}, Text: {row['text']}}}")

In [ ]:
# Сортируем по убыванию Factor 1 и берем первые 10 строк
top_10 = factor_scores.sort_values(by='Factor 6', ascending=False)[:50]

# Выводим тексты и значения, включая тип текста.

print("Тексты с наибольшими значениями Factor 6:")
for index, row in top_10.iterrows():
    print(f"Type: {row['type']}, Text: {row['text']}}}")



In [ ]:
# Сортируем по возрастанию Factor 1 и берем первые 10 строк
bottom_10 = factor_scores.sort_values(by='Factor 6', ascending=True)[:50]

# Выводим тексты и значения
# Выводим тексты и значения, включая тип текста
print("Тексты с наименьшими значениями Factor 6:")
for index, row in bottom_10.iterrows():
    if row['type']=='postcard':
        print(f"Type: {row['type']}, Text: {row['text']}}}")

In [ ]:
# Сортируем по убыванию Factor 1 и берем первые 10 строк
top_10 = factor_scores.sort_values(by='Factor 7', ascending=False)[:300]

# Выводим тексты и значения, включая тип текста
print("Тексты с наибольшими значениями Factor 7:")
for index, row in top_10.iterrows():
    if row['type'] == 'diary': 
        print(f"Type: {row['type']}, Text: {row['text']}}}")



In [ ]:
diary_count = top_10[top_10['type'] == 'diary']['type'].count()
postcard_count = top_10[top_10['type'] == 'postcard']['type'].count()

print(f"Количество дневников в топ-100: {diary_count}")
print(f"Количество открыток в топ-100: {postcard_count}")

In [ ]:
# Сортируем по возрастанию Factor 1 и берем первые 10 строк
bottom_10 = factor_scores.sort_values(by='Factor 7', ascending=True)[:100]

# Выводим тексты и значения
# Выводим тексты и значения, включая тип текста
print("Тексты с наименьшими значениями Factor 7:")
for index, row in bottom_10.iterrows():
    print(f"Type: {row['type']}, Text: {row['text']}}}")

In [ ]:
diary_count = bottom_10[bottom_10['type'] == 'diary']['type'].count()
postcard_count = bottom_10[bottom_10['type'] == 'postcard']['type'].count()

print(f"Количество дневников в топ-100: {diary_count}")
print(f"Количество открыток в топ-100: {postcard_count}")

In [ ]:
# Сортируем по убыванию Factor 1 и берем первые 10 строк
top_10 = factor_scores.sort_values(by='Factor 8', ascending=False)[:20]

# Выводим тексты и значения, включая тип текста
print("Тексты с наибольшими значениями Factor 8:")
for index, row in top_10.iterrows():
    print(f"Type: {row['type']}, Text: {row['text']}}}")



In [ ]:
# Сортируем по возрастанию Factor 1 и берем первые 10 строк
bottom_10 = factor_scores.sort_values(by='Factor 8', ascending=True)[:20]

# Выводим тексты и значения
# Выводим тексты и значения, включая тип текста
print("Тексты с наименьшими значениями Factor 8:")
for index, row in bottom_10.iterrows():
    if row['type'] == 'postcard':
        print(f"Type: {row['type']}, Text: {row['text']}}}")

In [ ]:
# Сортируем по убыванию Factor 1 и берем первые 10 строк
top_10 = factor_scores.sort_values(by='Factor 9', ascending=False)[:20]

# Выводим тексты и значения, включая тип текста
print("Тексты с наибольшими значениями Factor 9:")
for index, row in top_10.iterrows():
    print(f"Type: {row['type']}, Text: {row['text']}}}")



In [ ]:
# Сортируем по возрастанию Factor 1 и берем первые 10 строк
bottom_10 = factor_scores.sort_values(by='Factor 9', ascending=True)[:50]

# Выводим тексты и значения
# Выводим тексты и значения, включая тип текста
print("Тексты с наименьшими значениями Factor 9:")
for index, row in bottom_10.iterrows():
    print(f"Type: {row['type']}, Text: {row['text']}}}")

# Factor dinamics

In [ ]:
factors = [f'Factor {i}' for i in range(1, 10)]  # Список факторов

# Создаем фигуру с подграфиками
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(15, 10))
axes = axes.flatten()  # Преобразуем массив подграфиков в одномерный для удобства

# Перебираем факторы и создаем графики
for i, factor in enumerate(factors):
    sns.lineplot(data=factor_scores, x='decade', y=factor, ax=axes[i])
    axes[i].set_title(f'Динамика {factor} по декадам')
    axes[i].set_xlabel('Декада')
    axes[i].set_ylabel(factor)

plt.tight_layout() 
plt.show()


In [ ]:
factors = [f'Factor {i}' for i in range(1, 10)]

# Создаем фигуру с подграфиками
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(15, 10))
axes = axes.flatten()

# Перебираем факторы и создаем графики
for i, factor in enumerate(factors):
    sns.lineplot(data=factor_scores, x='decade', y=factor, hue='type', ax=axes[i]) #hue параметр
    axes[i].set_title(f'Динамика {factor} по декадам (Diary vs Postcard)')
    axes[i].set_xlabel('Декада')
    axes[i].set_ylabel(factor)

plt.tight_layout()
plt.show()



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

factors = [f'Factor {i}' for i in range(1, 10)]

# Создаем фигуру с подграфиками
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(15, 10))
axes = axes.flatten()

# Перебираем факторы и создаем графики
for i, factor in enumerate(factors):
    sns.lineplot(data=factor_scores, x='decade', y=factor, hue='type', ax=axes[i], palette='gray')  # Указание палитры
    axes[i].set_title(f'Динамика {factor} по декадам (Diary vs Postcard)')
    axes[i].set_xlabel('Декада')
    axes[i].set_ylabel(factor)

plt.tight_layout()
plt.show()


In [ ]:
factor_scores

In [ ]:
factor_means_by_decade = factor_scores.copy().drop(columns=['text', 'type'], errors='ignore')

# Группируем по декадам и вычисляем среднее
factor_means_by_decade = factor_means_by_decade.groupby('decade').mean(numeric_only=True)


# Выводим таблицу
factor_means_by_decade

In [ ]:
plt.figure(figsize=(12, 6))

# Для каждого фактора строим линию
for factor in factor_means_by_decade.columns:
    if factor.startswith('Factor'):
        plt.plot(
            factor_means_by_decade.index,
            factor_means_by_decade[factor],
            marker='o',
            label=factor
        )

plt.title('Динамика средних факторных оценок по декадам')
plt.xlabel('Десятилетие')
plt.ylabel('Средняя факторная оценка')
plt.legend()
plt.grid(True, which='both', linestyle='--', linewidth=0.7, color='gray', alpha=0.7)

plt.show()

In [ ]:
# Группировка по типу текста и декаде
factor_means_by_type_decade = factor_scores.copy().drop(columns=['text'], errors='ignore')

# Группировка и усреднение
factor_means_by_type_decade = factor_means_by_type_decade.groupby(['type', 'decade']).mean(numeric_only=True)

# Просмотр таблицы
factor_means_by_type_decade


In [ ]:
import matplotlib.pyplot as plt

# Получаем список факторов
factor_columns = [col for col in factor_means_by_type_decade.columns if col.startswith('Factor')]

# Получаем список типов текстов
text_types = factor_means_by_type_decade.index.get_level_values('type').unique()

for text_type in text_types:
    # Отбираем данные для конкретного типа
    data = factor_means_by_type_decade.loc[text_type]
    
    plt.figure(figsize=(12, 6))
    
    for factor in factor_columns:
        plt.plot(
            data.index,  # декады
            data[factor],  # значения фактора
            marker='o',
            label=factor
        )
    
    plt.title(f'Динамика средних факторных оценок по декадам для типа: {text_type}')
    plt.xlabel('Декада')
    plt.ylabel('Средняя факторная оценка')
    plt.legend()
    plt.grid(True, which='both', linestyle='--', linewidth=0.7, color='gray', alpha=0.7)
    
    plt.show()


In [ ]:
# from sklearn.cluster import KMeans
# from sklearn.preprocessing import StandardScaler

# # 1. Выделяем только факторные оценки (без года)
# data_for_clustering = cluster_by_factors[[col for col in cluster_by_factors.columns if col.startswith('Factor')]]

# # 2. Стандартизация
# scaler = StandardScaler()
# data_scaled = scaler.fit_transform(data_for_clustering)

# # 4. Кластеризация с оптимальным k (например, k=3)
# kmeans = KMeans(n_clusters=2, random_state=42)
# clusters = kmeans.fit_predict(data_scaled)
# cluster_by_factors['Период'] = clusters

# # 5. Визуализация по годам
# plt.figure(figsize=(12, 6))
# sns.boxplot(x='Период', y='year', data=cluster_by_factors)
# plt.title('Распределение годов по кластерам')
# plt.show()

# Temporal classification

## Data preparation

In [ ]:
counts = result_df.groupby(['decade', 'type']).size()
print("Исходное распределение:\n", counts)

In [ ]:
bins_20 = list(range(1900, 2041, 20))  # от 1900 до 2020 с шагом 20 лет
labels_20 = [f'{start}-{start+19}' for start in bins_20[:-1]]  # метки для интервалов
result_df['decade_class_20yrs'] = pd.cut(result_df['decade'], bins=bins_20, labels=labels_20, right=False)


In [ ]:
result_df

In [ ]:
# Указанные интервалы-исключения: последние два 20-летия
special_classes = ['2000-2019', '2020-2039']

# Создаем новый столбец для классификации по 20-летиям
result_df['decade_class_20yrs'] = pd.cut(result_df['decade'], bins=bins_20, labels=labels_20, right=False)

# Разделяем датафрейм
df_rest = result_df[~result_df['decade_class_20yrs'].isin(special_classes)]
df_special = result_df[result_df['decade_class_20yrs'].isin(special_classes)]

# Уровень выборки: 1600 для каждой 20-летней группы
balanced_samples = []

# Получаем список уникальных классов из df_rest
classes_20yrs = df_rest['decade_class_20yrs'].unique()

for cls in classes_20yrs:
    subset = df_rest[df_rest['decade_class_20yrs'] == cls]
    n_samples = min(1600, len(subset))  # на случай, если текстов меньше
    sampled = subset.sample(n=n_samples, random_state=42)
    balanced_samples.append(sampled)

# Добавляем все из special_classes без изменений
balanced_samples.append(df_special)

# Объединяем
df_balanced = pd.concat(balanced_samples, ignore_index=True)

# Выводим распределение
print("\nРаспределение после выравнивания:\n", 
      df_balanced['decade_class_20yrs'].value_counts().sort_index())


In [ ]:
# Классы 20-летий, которые нужно оставить полностью
special_classes = ['2000-2019', '2020-2039']

# Создаем новый столбец с делением по 20-летиям
result_df['decade_class_20yrs'] = pd.cut(result_df['decade'], bins=bins_20, labels=labels_20, right=False)

# Функция для создания сбалансированного датафрейма для одного типа
def create_balanced_df(df, text_type):
    df_type = df[df['type'] == text_type]
    
    df_rest = df_type[~df_type['decade_class_20yrs'].isin(special_classes)]
    df_special = df_type[df_type['decade_class_20yrs'].isin(special_classes)]
    
    balanced_samples = []
    
    classes_20yrs = df_rest['decade_class_20yrs'].unique()
    
    for cls in classes_20yrs:
        subset = df_rest[df_rest['decade_class_20yrs'] == cls]
        n_samples = min(1000, len(subset))  # чтобы не было ошибки при нехватке текстов
        sampled = subset.sample(n=n_samples, random_state=42)
        balanced_samples.append(sampled)
    
    # Добавляем все из special_classes без изменений
    balanced_samples.append(df_special)
    
    # Объединяем
    df_balanced = pd.concat(balanced_samples, ignore_index=True)
    
    return df_balanced

# Создаем два отдельных датафрейма
postcard_balanced = create_balanced_df(result_df, 'postcard')
diary_balanced = create_balanced_df(result_df, 'diary')

# Проверка распределения
print("\nРаспределение для postcard:\n", postcard_balanced['decade_class_20yrs'].value_counts().sort_index())
print("\nРаспределение для diary:\n", diary_balanced['decade_class_20yrs'].value_counts().sort_index())


In [ ]:
descriptive_stats = df_balanced.describe().T[['mean', 'min', 'max', 'std']]
descriptive_stats['range'] = descriptive_stats['max'] - descriptive_stats['min']

# Сохраняем в таблицу
descriptive_stats.to_csv('descriptive_stats.csv')
print(descriptive_stats)

In [ ]:
for col in nan_cols:
    df_balanced[col] = df_balanced[col].fillna(0).astype('float64')

nan_cols = df_balanced.columns[df_balanced.isnull().any()]
print("Столбцы, содержащие NaN:", nan_cols)

df_with_nan = df_balanced[nan_cols]

# Создаем булеву маску, где True соответствует строкам, содержащим хотя бы один NaN
rows_with_nan = df_with_nan.isnull().any(axis=1)

# Считаем количество строк, содержащих хотя бы один NaN
print(rows_with_nan.sum())


In [ ]:
standardized_feature_matrix  = df_balanced.copy()
standardized_feature_matrix

In [ ]:
standardized_feature_matrix  = standardized_feature_matrix.drop(['text', 'year', 'decade', 'type', 'text_length', 'decade_class_20yrs'], axis=1)
standardized_feature_matrix

In [ ]:
for column in standardized_feature_matrix.columns:
    mean = descriptive_stats.loc[column, 'mean']
    std = descriptive_stats.loc[column, 'std']

    standardized_feature_matrix[column] = (standardized_feature_matrix[column] - mean) / std
standardized_feature_matrix

In [ ]:
feature_columns_new = [col for col in standardized_feature_matrix.columns]
len(feature_columns_new)

In [ ]:
# Среднее после стандартизации должно быть ~0
print(standardized_feature_matrix[feature_columns_new].mean().round(2))  

# Стандартное отклонение должно быть ~1
print(standardized_feature_matrix[feature_columns_new].std().round(2))   

In [ ]:
# 1. Создаем DataFrame factor_scores с текстами и декадами
factor_scores = df_balanced[['text', 'decade', 'type', 'decade_class_20yrs']].copy()

# 2. Связываем standardized_feature_matrix с factor_scores по индексам
standardized_features = standardized_feature_matrix.copy()
standardized_features.index = factor_scores.index

In [ ]:
# 3. Функция для расчета факторной оценки
def calculate_factor_score(row, factor_features):
    score = 0
    for feature, loading in factor_features:
        # Получаем стандартизированное значение признака
        value = row[feature]
        # Учитываем знак нагрузки: + если loading > 0, - если loading < 0
        score += value * (1 if loading > 0 else -1)
    return score

In [ ]:
# 4. Для каждого фактора из factor_dict вычисляем оценку
for factor, features in factor_dict.items():
    # Признаки и их нагрузки для текущего фактора
    factor_features = [(feat[0], feat[1]) for feat in features]

    # Проверяем наличие признаков в standardized_features
    missing = [feat[0] for feat in features if feat[0] not in standardized_features.columns]
    if missing:
        print(f"Предупреждение: Признаки {missing} отсутствуют в standardized_feature_matrix.")
        continue

    # Рассчитываем факторную оценку
    factor_scores[factor] = standardized_features.apply(
        lambda row: calculate_factor_score(row, factor_features), axis=1
    )

factor_scores.head()

In [ ]:
# import pandas as pd
# factor_scores = pd.read_csv('/kaggle/input/normal-data/factor_scores.csv', index_col='Unnamed: 0').drop(columns=['decade_class'])

## Classification

In [ ]:
# Перемешиваем данные
factor_scores_shuffled = factor_scores.sample(frac=1, random_state=42).reset_index(drop=True)

# Проверяем результат
factor_scores_shuffled

In [ ]:
print("\nРаспределение после выравнивания:\n", 
      factor_scores_shuffled.groupby(['decade_class_20yrs']).size())


In [ ]:
from sklearn.preprocessing import LabelEncoder

# encoder = LabelEncoder()
# factor_scores_shuffled["decade_binary_encoded"] = encoder.fit_transform(factor_scores_shuffled["decade_class_binary"])
# target_decade = factor_scores_shuffled["decade_binary_encoded"].values

encoder_20yrs = LabelEncoder()
factor_scores_shuffled["decade_20yrs_encoded"] = encoder_20yrs.fit_transform(factor_scores_shuffled["decade_class_20yrs"])
target_20yrs = factor_scores_shuffled["decade_20yrs_encoded"].values


# encoder_10yrs = LabelEncoder()
# factor_scores_shuffled["decade_encoded"] = encoder_10yrs.fit_transform(factor_scores_shuffled["decade"])
# target_10yrs = factor_scores_shuffled["decade_encoded"].values


In [ ]:
# Выделяем только факторные признаки
factor_features_df = factor_scores_shuffled.filter(regex="Factor").copy()

# Сохраняем имена колонок
factor_columns = factor_features_df.columns.tolist()

# Получаем значения признаков
factor_features = factor_features_df.values


## 20s classification

In [ ]:
from sklearn.model_selection import train_test_split
X_train_full_20s, X_test_20s, y_train_full_20s, y_test_20s = train_test_split(
    factor_features, target_20yrs, 
    test_size=0.2, 
    stratify=target_20yrs, 
    random_state=42
)


X_train_20s, X_val_20s, y_train_20s, y_val_20s = train_test_split(
    X_train_full_20s, y_train_full_20s, 
    test_size=0.2, 
    stratify=y_train_full_20s, 
    random_state=42
)

### Model comparison

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier,
    ExtraTreesClassifier,
    GradientBoostingClassifier,
    HistGradientBoostingClassifier
)
from sklearn.metrics import classification_report, f1_score
import lightgbm as lgb
import xgboost as xgb
import numpy as np

# Инициализация моделей
tree_models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'Extra Trees': ExtraTreesClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'Hist Gradient Boosting': HistGradientBoostingClassifier(random_state=42),
    'LightGBM': lgb.LGBMClassifier(random_state=42),
    'XGBoost': xgb.XGBClassifier(random_state=42)
}

f1_scores_dict = {}

for name, model in tree_models.items():
    print(f"\n{'='*30} {name} {'='*30}")
    
    # Кросс-валидация на тренировочной выборке
    f1_scores = cross_val_score(model, X_train_20s, y_train_20s,
                                scoring='f1_macro', cv=5, n_jobs=-1)
    mean_f1 = np.mean(f1_scores)
    f1_scores_dict[name] = mean_f1
    print(f"Cross-validated macro-F1: {mean_f1:.3f}")

# Определим лучшую модель
best_model_name = max(f1_scores_dict, key=f1_scores_dict.get)
print(f"\n Лучшая модель по cross-val F1_macro: {best_model_name} ({f1_scores_dict[best_model_name]:.3f})")

# Инициализация лучшей модели
best_model = tree_models[best_model_name]


### Grid search

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, f1_score

# Инициализация модели
rf = RandomForestClassifier(random_state=42)

# Сетка гиперпараметров
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'max_features': ['sqrt', 0.8],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]
}

# GridSearchCV с метрикой macro F1
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=2
)

# Обучение
grid_search.fit(X_train_20s, y_train_20s)

# Лучшая модель и её гиперпараметры
print("\nЛучшие параметры:", grid_search.best_params_)
best_model = grid_search.best_estimator_

# Оценка на валидационной выборке
val_pred = best_model.predict(X_val_20s)
print("\nValidation Classification Report:")
print(classification_report(y_val_20s, val_pred, digits=3))

# Оценка на тестовой выборке
test_pred = best_model.predict(X_test_20s)
print("\nTest Classification Report:")
print(classification_report(y_test_20s, test_pred, digits=3))

print(f"\nMacro-F1 (Validation): {f1_score(y_val_20s, val_pred, average='macro'):.3f}")
print(f"Macro-F1 (Test): {f1_score(y_test_20s, test_pred, average='macro'):.3f}")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
best_model = RandomForestClassifier(min_samples_leaf=2, n_estimators=200, random_state=42)


### Training best model

In [ ]:
best_model.fit(X_train_20s, y_train_20s)


y_val_pred = best_model.predict(X_val_20s)
print("Classification report for postcards (Validation):")
print(classification_report(y_val_20s, y_val_pred, digits=3))
macro_f1_val = f1_score(y_val_20s, y_val_pred, average='macro')
print(f"Macro F1 for postcards (Validation): {macro_f1_val:.3f}")

In [ ]:
pd.set_option('display.precision', 3)
importances = best_model.feature_importances_

feature_importance_df = pd.DataFrame({'Feature': factor_columns, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
feature_importance_df


In [ ]:
pip install shap


In [ ]:
import shap

# Инициализация Explainer
explainer = shap.TreeExplainer(best_model)

# Вычисление значений SHAP
shap_values = explainer.shap_values(X_val_20s)  # X_val — валидационные признаки (без таргета)


In [ ]:
import matplotlib.pyplot as plt



# Создаем сетку графиков
n_classes = len(shap_values)
n_cols = 3  # Колонок в ряду
n_rows = (n_classes + n_cols - 1) // n_cols  # Вычисляем нужное количество рядов

plt.figure(figsize=(n_cols*6, n_rows*4))  # Размер фигуры

for i in range(n_classes):
    plt.subplot(n_rows, n_cols, i+1)  # Позиция графика в сетке
    shap.summary_plot(
        shap_values[i], 
        X_val_20s, 
        cmap=plt.cm.Greys, # Градиент от белого к черному
        feature_names=factor_columns,
        show=False,  # Не показывать сразу
        plot_size=None  # Отключить авторазмер
    )
    plt.title(f"Class {i}", fontsize=12)
    plt.gcf().tight_layout()  # Оптимизация расположения

plt.show()

In [ ]:
import matplotlib.pyplot as plt



# Создаем сетку графиков
n_classes = len(shap_values)
n_cols = 3  # Колонок в ряду
n_rows = (n_classes + n_cols - 1) // n_cols  # Вычисляем нужное количество рядов

plt.figure(figsize=(n_cols*6, n_rows*4))  # Размер фигуры

for i in range(n_classes):
    plt.subplot(n_rows, n_cols, i+1)  # Позиция графика в сетке
    shap.summary_plot(
        shap_values[i], 
        X_val_20s,  
        feature_names=factor_columns,
        show=False,  # Не показывать сразу
        plot_size=None  # Отключить авторазмер
    )
    plt.title(f"Class {i}", fontsize=12)
    plt.gcf().tight_layout()  # Оптимизация расположения

plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import f1_score

# Распределение классов в обучающих данных
class_counts = np.bincount(y_train_20s)
class_probabilities = class_counts / len(y_train_20s)
classes = np.arange(len(class_probabilities))

# Количество прогонов для оценки матожидания
n_runs = 1000
macro_f1_scores = []

for _ in range(n_runs):
    # Случайные предсказания согласно распределению классов
    y_pred_random = np.random.choice(classes, size=len(y_test_20s), p=class_probabilities)
    
    # Расчет macro-F1
    f1 = f1_score(y_test_20s, y_pred_random, average="macro")
    macro_f1_scores.append(f1)

# Ожидаемый macro-F1
expected_macro_f1 = np.mean(macro_f1_scores)
print(f"Ожидаемый Macro-F1: {expected_macro_f1:.4f}")

In [ ]:
most_frequent_class = np.argmax(class_probabilities)
y_pred_constant = np.full(len(y_test_20s), most_frequent_class)

constant_macro_f1 = f1_score(y_test_20s, y_pred_constant, average="macro")
print(f"Macro-F1 константного классификатора: {constant_macro_f1:.4f}")

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Построение матрицы ошибок
cm = confusion_matrix(y_val_20s, val_pred)

# Нормализация по строкам и перевод в проценты
cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

# Подписи классов
decade_labels = [str(decade) for decade in encoder_20yrs.classes_]

# Визуализация
plt.figure(figsize=(8, 6))
sns.heatmap(cm_percent, annot=True, fmt='.1f', cmap='Greys',
            xticklabels=decade_labels, yticklabels=decade_labels,
            cbar_kws={'label': 'Percentage (%)'})

plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("Confusion Matrix (Percent, Validation Set)")
plt.show()


## Models for register types 

In [ ]:
print("\nРаспределение для postcard:\n", postcard_balanced['decade_class_20yrs'].value_counts().sort_index())
print("\nРаспределение для diary:\n", diary_balanced['decade_class_20yrs'].value_counts().sort_index())

### postcards

In [ ]:
descriptive_stats_pt = postcard_balanced.describe().T[['mean', 'min', 'max', 'std']]
descriptive_stats_pt['range'] = descriptive_stats_pt['max'] - descriptive_stats_pt['min']

# Сохраняем в таблицу
descriptive_stats_pt.to_csv('descriptive_stats.csv')
print(descriptive_stats_pt)

In [ ]:
for col in nan_cols:
    postcard_balanced[col] = postcard_balanced[col].fillna(0).astype('float64')

nan_cols = postcard_balanced.columns[postcard_balanced.isnull().any()]
print("Столбцы, содержащие NaN:", nan_cols)

df_with_nan = postcard_balanced[nan_cols]

# Создаем булеву маску, где True соответствует строкам, содержащим хотя бы один NaN
rows_with_nan = df_with_nan.isnull().any(axis=1)

# Считаем количество строк, содержащих хотя бы один NaN
print(rows_with_nan.sum())


In [ ]:
standardized_feature_matrix_pt  = postcard_balanced.copy()
standardized_feature_matrix_pt

In [ ]:
standardized_feature_matrix_pt  = standardized_feature_matrix_pt.drop(['text', 'year', 'decade', 'type', 'text_length', 'decade_class_20yrs'], axis=1)
standardized_feature_matrix_pt

In [ ]:
for column in standardized_feature_matrix_pt.columns:
    mean = descriptive_stats_pt.loc[column, 'mean']
    std = descriptive_stats_pt.loc[column, 'std']

    standardized_feature_matrix_pt[column] = (standardized_feature_matrix_pt[column] - mean) / std
standardized_feature_matrix_pt

In [ ]:
# Среднее после стандартизации должно быть ~0
print(standardized_feature_matrix_pt[feature_columns_new].mean().round(2))  

# Стандартное отклонение должно быть ~1
print(standardized_feature_matrix_pt[feature_columns_new].std().round(2))   

In [ ]:
# 1. Создаем DataFrame factor_scores с текстами и декадами
factor_scores_pt = postcard_balanced[['text', 'decade', 'type', 'decade_class_20yrs']].copy()

# 2. Связываем standardized_feature_matrix с factor_scores по индексам
standardized_features_pt = standardized_feature_matrix_pt.copy()
standardized_features_pt.index = factor_scores_pt.index

In [ ]:
# 4. Для каждого фактора из factor_dict вычисляем оценку
for factor, features in factor_dict.items():
    # Признаки и их нагрузки для текущего фактора
    factor_features = [(feat[0], feat[1]) for feat in features]

    # Проверяем наличие признаков в standardized_features
    missing = [feat[0] for feat in features if feat[0] not in standardized_features_pt.columns]
    if missing:
        print(f"Предупреждение: Признаки {missing} отсутствуют в standardized_feature_matrix.")
        continue

    # Рассчитываем факторную оценку
    factor_scores_pt[factor] = standardized_features_pt.apply(
        lambda row: calculate_factor_score(row, factor_features), axis=1
    )

factor_scores_pt

In [ ]:
# Перемешиваем данные
factor_scores_shuffled_pt = factor_scores_pt.sample(frac=1, random_state=42).reset_index(drop=True)

# Проверяем результат
factor_scores_shuffled_pt

In [ ]:
encoder_20yrs = LabelEncoder()
factor_scores_shuffled_pt["decade_20yrs_encoded"] = encoder_20yrs.fit_transform(factor_scores_shuffled_pt["decade_class_20yrs"])
target_20yrs_pt = factor_scores_shuffled_pt["decade_20yrs_encoded"].values

In [ ]:
# Выделяем только факторные признаки
factor_features_df_pt = factor_scores_shuffled_pt.filter(regex="Factor").copy()

# Сохраняем имена колонок
factor_columns = factor_features_df_pt.columns.tolist()

# Получаем значения признаков
factor_features_pt = factor_features_df_pt.values


In [ ]:
from sklearn.model_selection import train_test_split
X_train_full_20s_pt, X_test_20s_pt, y_train_full_20s_pt, y_test_20s_pt = train_test_split(
    factor_features_pt, target_20yrs_pt, 
    test_size=0.2, 
    stratify=target_20yrs_pt, 
    random_state=42
)


X_train_20s_pt, X_val_20s_pt, y_train_20s_pt, y_val_20s_pt = train_test_split(
    X_train_full_20s_pt, y_train_full_20s_pt, 
    test_size=0.2, 
    stratify=y_train_full_20s_pt, 
    random_state=42
)

In [ ]:
# Инициализация моделей с одинаковыми параметрами
rf_postcards = RandomForestClassifier(min_samples_leaf=2, n_estimators=200, random_state=42)

# Обучение модели для открыток
rf_postcards.fit(X_train_20s_pt, y_train_20s_pt)


y_val_pred_postcards = rf_postcards.predict(X_val_20s_pt)
print("Classification report for postcards (Validation):")
print(classification_report(y_val_20s_pt, y_val_pred_postcards, digits=3))
macro_f1_postcards_val = f1_score(y_val_20s_pt, y_val_pred_postcards, average='macro')
print(f"Macro F1 for postcards (Validation): {macro_f1_postcards_val:.3f}")

In [ ]:
pd.set_option('display.precision', 3)
importances = rf_postcards.feature_importances_

feature_importance_df = pd.DataFrame({'Feature': factor_columns, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
feature_importance_df


### diaries 

In [ ]:
descriptive_stats_pr = diary_balanced.describe().T[['mean', 'min', 'max', 'std']]
descriptive_stats_pr['range'] = descriptive_stats_pr['max'] - descriptive_stats_pr['min']

# Сохраняем в таблицу
descriptive_stats_pr.to_csv('descriptive_stats.csv')
print(descriptive_stats_pr)

In [ ]:
for col in nan_cols:
    diary_balanced[col] = diary_balanced[col].fillna(0).astype('float64')

nan_cols = diary_balanced.columns[diary_balanced.isnull().any()]
print("Столбцы, содержащие NaN:", nan_cols)

df_with_nan = diary_balanced[nan_cols]

# Создаем булеву маску, где True соответствует строкам, содержащим хотя бы один NaN
rows_with_nan = df_with_nan.isnull().any(axis=1)

# Считаем количество строк, содержащих хотя бы один NaN
print(rows_with_nan.sum())


In [ ]:
standardized_feature_matrix_pr  = diary_balanced.copy()
standardized_feature_matrix_pr

In [ ]:
standardized_feature_matrix_pr  = standardized_feature_matrix_pr.drop(['text', 'year', 'decade', 'type', 'text_length', 'decade_class_20yrs'], axis=1)
standardized_feature_matrix_pr

In [ ]:
for column in standardized_feature_matrix_pr.columns:
    mean = descriptive_stats_pr.loc[column, 'mean']
    std = descriptive_stats_pr.loc[column, 'std']

    standardized_feature_matrix_pr[column] = (standardized_feature_matrix_pr[column] - mean) / std
standardized_feature_matrix_pr

In [ ]:
# Среднее после стандартизации должно быть ~0
print(standardized_feature_matrix_pr[feature_columns_new].mean().round(2))  

# Стандартное отклонение должно быть ~1
print(standardized_feature_matrix_pr[feature_columns_new].std().round(2))   

In [ ]:
# 1. Создаем DataFrame factor_scores с текстами и декадами
factor_scores_pr = diary_balanced[['text', 'decade', 'type', 'decade_class_20yrs']].copy()

# 2. Связываем standardized_feature_matrix с factor_scores по индексам
standardized_features_pr = standardized_feature_matrix_pr.copy()
standardized_features_pr.index = factor_scores_pr.index

In [ ]:
# 4. Для каждого фактора из factor_dict вычисляем оценку
for factor, features in factor_dict.items():
    # Признаки и их нагрузки для текущего фактора
    factor_features = [(feat[0], feat[1]) for feat in features]

    # Проверяем наличие признаков в standardized_features
    missing = [feat[0] for feat in features if feat[0] not in standardized_features_pr.columns]
    if missing:
        print(f"Предупреждение: Признаки {missing} отсутствуют в standardized_feature_matrix.")
        continue

    # Рассчитываем факторную оценку
    factor_scores_pr[factor] = standardized_features_pr.apply(
        lambda row: calculate_factor_score(row, factor_features), axis=1
    )

factor_scores_pr

In [ ]:
# Перемешиваем данные
factor_scores_shuffled_pr = factor_scores_pr.sample(frac=1, random_state=42).reset_index(drop=True)

# Проверяем результат
factor_scores_shuffled_pr

In [ ]:
encoder_20yrs = LabelEncoder()
factor_scores_shuffled_pr["decade_20yrs_encoded"] = encoder_20yrs.fit_transform(factor_scores_shuffled_pr["decade_class_20yrs"])
target_20yrs_pr = factor_scores_shuffled_pr["decade_20yrs_encoded"].values

In [ ]:
# Выделяем только факторные признаки
factor_features_df_pr = factor_scores_shuffled_pr.filter(regex="Factor").copy()

# Сохраняем имена колонок
factor_columns = factor_features_df_pr.columns.tolist()

# Получаем значения признаков
factor_features_pr = factor_features_df_pr.values


In [ ]:
from sklearn.model_selection import train_test_split
X_train_full_20s_pr, X_test_20s_pr, y_train_full_20s_pr, y_test_20s_pr = train_test_split(
    factor_features_pr, target_20yrs_pr, 
    test_size=0.2, 
    stratify=target_20yrs_pr, 
    random_state=42
)


X_train_20s_pr, X_val_20s_pr, y_train_20s_pr, y_val_20s_pr = train_test_split(
    X_train_full_20s_pr, y_train_full_20s_pr, 
    test_size=0.2, 
    stratify=y_train_full_20s_pr, 
    random_state=42
)

In [ ]:
# Инициализация моделей с одинаковыми параметрами
rf_diaries = RandomForestClassifier(min_samples_leaf=2, n_estimators=200, random_state=42)

# Обучение модели для открыток
rf_diaries.fit(X_train_20s_pr, y_train_20s_pr)


y_val_pred_diaries = rf_diaries.predict(X_val_20s_pr)
print("Classification report for postcards (Validation):")
print(classification_report(y_val_20s_pr, y_val_pred_diaries, digits=3))
macro_f1_diaires_val = f1_score(y_val_20s_pr, y_val_pred_diaries, average='macro')
print(f"Macro F1 for postcards (Validation): {macro_f1_diaires_val:.3f}")

In [ ]:
pd.set_option('display.precision', 3)
importances = rf_diaries.feature_importances_

feature_importance_df = pd.DataFrame({'Feature': factor_columns, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
feature_importance_df


----------------------